In [1]:
import os
import numpy as np
import librosa
import noisereduce as nr
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
!pip install torch torchaudio transformers tqdm

In [3]:
from tqdm import tqdm
import torchaudio
from transformers import Wav2Vec2Processor, HubertForSequenceClassification
# Load the pretrained HuBERT model and processor
from transformers import AutoFeatureExtractor, HubertForSequenceClassification

In [4]:
!pip install torch torchvision torchaudio

In [5]:
import torch


In [6]:
import os
import numpy as np
import librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
import os
from tqdm import tqdm
import librosa
import torch
from sklearn.model_selection import train_test_split
from transformers import Wav2Vec2Processor, HubertModel, ViTConfig, ViTForImageClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch.nn as nn
import torch.optim as optim

In [10]:
import os
import numpy as np

# Function to recursively collect .wav files and their labels
def load_dataset(base_path):
    file_paths = []
    labels = []
    categories = {'Bonafide': 1, 'Spoofed_TTS': 0, 'Spoofed_Tacotron': 0}  # Map categories to labels

    # Debug: Check if the base path exists
    if not os.path.exists(base_path):
        print(f"Error: The base path {base_path} does not exist.")
        return file_paths, labels

    for category, label in categories.items():
        category_path = os.path.join(base_path, category)

        # Debug: Check if the category folder exists
        if not os.path.exists(category_path):
            print(f"Error: The category folder {category_path} does not exist.")
            continue  # Skip this category if the folder is missing

        print(f"Processing category: {category}...")

        for speaker_folder in os.listdir(category_path):
            speaker_path = os.path.join(category_path, speaker_folder)

            # Debug: Check if the folder is a directory
            if os.path.isdir(speaker_path):
                print(f"  Processing speaker folder: {speaker_folder}...")
                # Special case for Bonafide: Handle nested Part 1 and Part 2 folders
                if category == 'Bonafide':
                    for part_folder in os.listdir(speaker_path):
                        part_path = os.path.join(speaker_path, part_folder)
                        if os.path.isdir(part_path):
                            print(f"    Processing part folder: {part_folder}...")
                            for file_name in os.listdir(part_path):
                                if file_name.endswith('.wav'):
                                    file_path = os.path.join(part_path, file_name)
                                    file_paths.append(file_path)
                                    labels.append(label)
                else:
                    # Handle non-nested folders for Spoofed_TTS and Spoofed_Tacotron
                    for file_name in os.listdir(speaker_path):
                        if file_name.endswith('.wav'):
                            file_path = os.path.join(speaker_path, file_name)
                            file_paths.append(file_path)
                            labels.append(label)
            else:
                print(f"  Skipping non-directory: {speaker_folder}")

    return np.array(file_paths), np.array(labels)

# Set the base path where your dataset is stored
base_path = r'D:\fast\fyp\DATASET'  # Update path to your dataset location

# Load dataset
file_paths, labels = load_dataset(base_path)

# Display the loaded data for debugging
print(f"Total audio files: {len(file_paths)}")
print(f"First 5 file paths: {file_paths[:5]}")
print(f"First 5 labels: {labels[:5]}")


Processing category: Bonafide...
  Processing speaker folder: Speaker_01...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_02...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_03...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_04...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_05...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_06...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_07...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing speaker folder: Speaker_08...
    Processing part folder: Part 1...
    Processing part folder: Part 2...
  Processing sp

In [12]:
def list_audio_files(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.wav')]

# Organize dataset into categories
audio_files = {'Bonafide': [], 'Spoofed_TTS': [], 'Spoofed_Tacotron': []}
for category in ['Bonafide', 'Spoofed_TTS', 'Spoofed_Tacotron']:
    category_path = os.path.join(base_path, category)

    for speaker_folder in os.listdir(category_path):
        speaker_path = os.path.join(category_path, speaker_folder)

        # Handle Bonafide folder with Part 1 and Part 2 subfolders
        if category == 'Bonafide':
            for part in ['Part 1', 'Part 2']:
                part_path = os.path.join(speaker_path, part)
                audio_files[category].extend(list_audio_files(part_path))
        else:
            audio_files[category].extend(list_audio_files(speaker_path))

In [13]:
c

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

C:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FAST\.cache\huggingface\hub\models--facebook--hubert-large-ls960-ft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Extracting HuBERT embeddings:   0%|                                                           | 0/5435 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\transformers\models\hubert\modeling_hubert.py:762: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Extracting HuBERT embeddings: 100%|████████████████████████████████████████████████| 1359/1359 [01:28<00:00, 15.39it/s]


In [16]:
import torch

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [19]:
import numpy as np
import math

# Correct embedding length calculation
embedding_length = x_train.shape[1]  # Length of each embedding vector
print(f"Embedding length: {embedding_length}")

# Ensure target size is the next perfect square if needed
target_size = math.ceil(math.sqrt(embedding_length)) ** 2
height = width = int(math.sqrt(target_size))

# Debugging target size
print(f"Target size: {target_size}")
print(f"Height and Width: {height}, {width}")

# Function to pad embeddings to make them square
def pad_to_square(embeddings, target_size):
    padded_embeddings = []
    for embedding in embeddings:
        # Padding to match the target size
        padding_length = target_size - embedding.shape[0]
        padded_embedding = np.pad(embedding, (0, padding_length), mode="constant")
        padded_embeddings.append(padded_embedding.reshape(height, width))  # Reshape to 2D
    return np.array(padded_embeddings)

# Pad and reshape embeddings for training and testing
x_train_square = pad_to_square(x_train, target_size)
x_test_square = pad_to_square(x_test, target_size)

# Verify shapes after padding and reshaping
print(f"x_train_square shape: {x_train_square.shape}")  # Expected: (n_samples, height, width)
print(f"x_test_square shape: {x_test_square.shape}")    # Expected: (n_samples, height, width)


Embedding length: 1024
Target size: 1024
Height and Width: 32, 32
x_train_square shape: (5435, 32, 32)
x_test_square shape: (1359, 32, 32)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# Custom Dataset class
class AudioDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Prepare Dataset
train_dataset = AudioDataset(x_train_square, y_train)
test_dataset = AudioDataset(x_test_square, y_test)

# DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [21]:
# Define Vision Transformer Model
class ViTForAudio(nn.Module):
    def __init__(self, input_dim, num_classes, patch_size=4, num_layers=6, num_heads=8, hidden_dim=256):
        super(ViTForAudio, self).__init__()
        self.input_dim = input_dim
        self.patch_size = patch_size
        self.num_classes = num_classes
        
        # Transformer encoder
        self.embedding = nn.Conv2d(1, hidden_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_encoding = nn.Parameter(torch.randn(1, (input_dim // patch_size) ** 2, hidden_dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads), num_layers=num_layers
        )
        self.classifier = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.embedding(x)  # (B, hidden_dim, H/patch, W/patch)
        x = x.flatten(2).transpose(1, 2)  # (B, num_patches, hidden_dim)
        x = x + self.positional_encoding  # Add positional encoding
        x = self.transformer(x)  # Transformer encoding
        x = x.mean(dim=1)  # Global average pooling
        x = self.classifier(x)  # Classification
        return x

# Initialize the Model
input_dim = x_train_square.shape[1]  # Height/Width of reshaped embeddings
num_classes = 2
model = ViTForAudio(input_dim, num_classes).to("cuda")

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [22]:
# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for embeddings, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        embeddings, labels = embeddings.to("cuda"), labels.to("cuda")
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for embeddings, labels in test_loader:
        embeddings, labels = embeddings.to("cuda"), labels.to("cuda")
        outputs = model(embeddings)
        predictions = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Compute Metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Epoch 1/10: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:04<00:00, 41.28it/s]


Epoch 1: Train Loss = 0.7498


Epoch 2/10: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 45.15it/s]


Epoch 2: Train Loss = 0.7013


Epoch 3/10: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 45.28it/s]


...

Epoch 8: Train Loss = 0.6940


Epoch 9/10: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 44.40it/s]


Epoch 9: Train Loss = 0.6946


Epoch 10/10: 100%|███████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 44.36it/s]


Epoch 10: Train Loss = 0.6938
Accuracy: 0.5011
F1 Score: 0.0000


FINE TUNED VIT


In [ ]:
# Fine-tuning Vision Transformer

# Adjusting ViT model with dropout for regularization
class ViTForAudioFineTuned(nn.Module):
    def __init__(self, input_dim, num_classes, patch_size=4, num_layers=6, num_heads=8, hidden_dim=256, dropout=0.1):
        super(ViTForAudioFineTuned, self).__init__()
        self.input_dim = input_dim
        self.patch_size = patch_size
        self.num_classes = num_classes
        
        self.embedding = nn.Conv2d(1, hidden_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_encoding = nn.Parameter(torch.randn(1, (input_dim // patch_size) ** 2, hidden_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.embedding(x)  # (B, hidden_dim, H/patch, W/patch)
        x = x.flatten(2).transpose(1, 2)  # (B, num_patches, hidden_dim)
        x = x + self.positional_encoding  # Add positional encoding
        x = self.transformer(x)  # Transformer encoding
        x = x.mean(dim=1)  # Global average pooling
        x = self.classifier(x)  # Classification
        return x

# Re-initialize the model
fine_tuned_model = ViTForAudioFineTuned(input_dim, num_classes).to("cuda")

# Optimizer with weight decay and learning rate scheduler
optimizer = optim.AdamW(fine_tuned_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Loss function with class weights (if imbalance exists)
class_weights = torch.tensor([1.0, 1.0]).to("cuda")  # Adjust if imbalance is present
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Training loop with fine-tuning
num_epochs = 20
for epoch in range(num_epochs):
    fine_tuned_model.train()
    train_loss = 0
    for embeddings, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        embeddings, labels = embeddings.to("cuda"), labels.to("cuda")
        optimizer.zero_grad()
        outputs = fine_tuned_model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss / len(train_loader):.4f}")

# Evaluation with precision and F1 score
from sklearn.metrics import precision_score

fine_tuned_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for embeddings, labels in test_loader:
        embeddings, labels = embeddings.to("cuda"), labels.to("cuda")
        outputs = fine_tuned_model(embeddings)
        predictions = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")


# new fine tuned 

In [24]:
# Adjust class weights based on the dataset
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights = torch.tensor(class_weights).float().to("cuda")

criterion = nn.CrossEntropyLoss(weight=class_weights)


In [25]:
# Early stopping utility
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        torch.save(model.state_dict(), "checkpoint.pt")

# Initialize EarlyStopping
early_stopping = EarlyStopping(patience=5)


In [26]:
num_epochs = 20  # Increased epochs
for epoch in range(num_epochs):
    fine_tuned_model.train()
    train_loss = 0
    for embeddings, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        embeddings, labels = embeddings.to("cuda"), labels.to("cuda")
        optimizer.zero_grad()
        outputs = fine_tuned_model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation loop
    fine_tuned_model.eval()
    val_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for embeddings, labels in test_loader:
            embeddings, labels = embeddings.to("cuda"), labels.to("cuda")
            outputs = fine_tuned_model(embeddings)
            val_loss += criterion(outputs, labels).item()
            predictions = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    val_loss /= len(test_loader)
    scheduler.step()
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss / len(train_loader):.4f}, Val Loss = {val_loss:.4f}")

    # Check Early Stopping
    early_stopping(val_loss, fine_tuned_model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# Load the best model checkpoint
fine_tuned_model.load_state_dict(torch.load("checkpoint.pt"))


Epoch 1/20: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:04<00:00, 41.68it/s]


Epoch 1: Train Loss = 0.6930, Val Loss = 0.6932


Epoch 2/20: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 43.99it/s]


Epoch 2: Train Loss = 0.6931, Val Loss = 0.6932


Epoch 3/20: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 43.63it/s]


...

Epoch 7: Train Loss = 0.6939, Val Loss = 0.6932


Epoch 8/20: 100%|████████████████████████████████████████████████████████████████████| 170/170 [00:03<00:00, 43.44it/s]


Epoch 8: Train Loss = 0.6931, Val Loss = 0.6932
Early stopping triggered


<ipython-input-26-512266e72c3d>:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fine_tuned_model.load_state_dict(torch.load("checkpoint.pt"))


<All keys matched successfully>

In [27]:
from sklearn.metrics import confusion_matrix, classification_report

conf_matrix = confusion_matrix(y_true, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Confusion Matrix:
[[  0 681]
 [  0 678]]
Accuracy: 0.4989
F1 Score: 0.6657
Precision: 0.4989

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       681
           1       0.50      1.00      0.67       678

    accuracy                           0.50      1359
   macro avg       0.25      0.50      0.33      1359
weighted avg       0.25      0.50      0.33      1359



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# new approach

MLP


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, f1_score
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Define a fully connected neural network (MLP)
class MLPForEmbeddings(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=2):
        super(MLPForEmbeddings, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Step 2: Convert embeddings to torch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to('cuda')
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to('cuda')
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to('cuda')
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to('cuda')

# Step 3: Create data loaders
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 4: Initialize the model, loss function, and optimizer
input_dim = x_train.shape[1]  # Number of features (dimensions of embeddings)
model = MLPForEmbeddings(input_dim).to('cuda')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print training loss
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # Step 6: Evaluate on the test set
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for batch in test_loader:
            inputs, labels = batch
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Calculate accuracy, precision, and F1 score
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)

        print(f"Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")

# Step 7: Final evaluation on the test set
model.eval()
with torch.no_grad():
    all_preds = []
    all_labels = []
    for batch in test_loader:
        inputs, labels = batch
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    # Final metrics
    final_accuracy = accuracy_score(all_labels, all_preds)
    final_precision = precision_score(all_labels, all_preds)
    final_f1 = f1_score(all_labels, all_preds)

    print(f"Final Accuracy: {final_accuracy:.4f}, Final Precision: {final_precision:.4f}, Final F1 Score: {final_f1:.4f}")


Epoch 1/20, Train Loss: 0.5683
Epoch 1/20, Accuracy: 0.8300, Precision: 0.8331, F1 Score: 0.8288
Epoch 2/20, Train Loss: 0.3729
Epoch 2/20, Accuracy: 0.8882, Precision: 0.9188, F1 Score: 0.8836
Epoch 3/20, Train Loss: 0.2696
Epoch 3/20, Accuracy: 0.8896, Precision: 0.8359, F1 Score: 0.8975
Epoch 4/20, Train Loss: 0.2305
Epoch 4/20, Accuracy: 0.9323, Precision: 0.9234, F1 Score: 0.9328
Epoch 5/20, Train Loss: 0.1992
Epoch 5/20, Accuracy: 0.9411, Precision: 0.9164, F1 Score: 0.9427
Epoch 6/20, Train Loss: 0.1749
Epoch 6/20, Accuracy: 0.9514, Precision: 0.9527, F1 Score: 0.9513
Epoch 7/20, Train Loss: 0.1294
Epoch 7/20, Accuracy: 0.9610, Precision: 0.9685, F1 Score: 0.9606
Epoch 8/20, Train Loss: 0.1432
Epoch 8/20, Accuracy: 0.9551, Precision: 0.9303, F1 Score: 0.9563
Epoch 9/20, Train Loss: 0.1046
Epoch 9/20, Accuracy: 0.9603, Precision: 0.9727, F1 Score: 0.9596
Epoch 10/20, Train Loss: 0.1259
Epoch 10/20, Accuracy: 0.9669, Precision: 0.9803, F1 Score: 0.9663
Epoch 11/20, Train Loss: 0.1

# final error check: